In [ ]:
from google.colab import drive
drive.mount('/content/drive')

In [ ]:
!pip install git+https://github.com/KirillDeLutherSergeev/AmperiumNNM@main

In [ ]:
# EDIT THIS SECTION FOR USER INPUTS
#
name = '1959HW_V1-4_V2-2'
in_file = '/content/drive/MyDrive/ML/NAM_1959HW/v1_1_1.wav'
out_file = '/content/drive/MyDrive/ML/NAM_1959HW/output/1959HW_V1-4_V2-2.wav'

if not os.path.exists('models/'+name):
    os.makedirs('models/'+name)
else:
    print("A model with the same name already exists. Please choose a new name.")
    exit

# Load and Preprocess Data ###########################################
in_rate, in_data = wavfile.read(in_file)
out_rate, out_data = wavfile.read(out_file)

#offset = 5 * in_rate
offset = 56228
x_all = in_data.astype(np.float32).flatten()
x_all = x_all[offset:]  
x_all = normalize(x_all).reshape(len(x_all),1)   
y_all = out_data.astype(np.float32).flatten() 
#y_all = y_all[offset-2:]
y_all = y_all[offset:]
y_all = normalize(y_all).reshape(len(y_all),1)    

In [ ]:
train_size = 0.8

epochs = 8
num_steps = 2048
overlap = 256
batch_size = 64 
learning_rate = 0.008
epsilon = 1.e-08
hidden_units = 16

# Create Sequential Model ###########################################
clear_session()

model = Sequential()

model.add(InputLayer(input_shape=(None, 1)))

useD1 = True
useC1 = True
useC2 = True

if useD1:
  model.add(Dense(units=1, use_bias=False,
    kernel_initializer=tf.keras.initializers.GlorotUniform(seed=None),
    bias_initializer='zeros',
    activation='tanh',
    name='D1'))

if useC2:
  conv_init = np.zeros(128)
  conv_init[0] = 1.0
  model.add(Conv1D(filters=1, kernel_size=128,
    padding='causal',
    bias_initializer='zeros',
    kernel_initializer=tf.constant_initializer(conv_init),
    use_bias=False,
    name='C1'))

model.add(LSTM(name='L', units=hidden_units,
  stateful=False,
  bias_initializer='zeros',
  kernel_initializer=tf.keras.initializers.GlorotUniform(seed=None),
  recurrent_initializer=tf.keras.initializers.Orthogonal(seed=None),
  return_sequences=True))

model.add(Dense(units=1,
  kernel_initializer=tf.keras.initializers.GlorotUniform(seed=None),
  bias_initializer='zeros',
  name='D2'))

if useC2:
  conv_init = np.zeros(2048)
  conv_init[0] = 1.0
  model.add(Conv1D(filters=1, kernel_size=2048,
    padding='causal',
    bias_initializer='zeros',
    kernel_initializer=tf.constant_initializer(conv_init),
    use_bias=False,
    name='C2'))

#model.compile(optimizer=Adam(learning_rate=learning_rate, epsilon=epsilon), loss=error_to_signal, metrics='mse')
#model.compile(optimizer=Adam(learning_rate=learning_rate, epsilon=epsilon), loss=esr_loss_with_frequency, metrics='mse')
#model.compile(optimizer=Adam(learning_rate=learning_rate, epsilon=epsilon), loss=esr_loss, metrics='mse')
model.compile(optimizer=Adam(learning_rate=learning_rate, epsilon=epsilon), loss='mae', metrics='mse')
#model.build()
model.summary()

In [ ]:
x_train, y_train, x_test, y_test = partition_data(x_all,y_all,num_steps,train_size,overlap)

train_observations = int(x_train.shape[0] / batch_size) * batch_size
test_observations = int(x_test.shape[0] / batch_size) * batch_size
x_train = x_train[:train_observations]
y_train = y_train[:train_observations]
x_test = x_test[:test_observations]
y_test = y_test[:test_observations]
x_train = x_train.reshape(x_train.shape[0], x_train.shape[1], 1)
y_train = y_train.reshape(y_train.shape[0], y_train.shape[1], 1)
x_test = x_test.reshape(x_test.shape[0], x_test.shape[1], 1)
y_test = y_test.reshape(y_test.shape[0], y_test.shape[1], 1)

In [ ]:
def scheduler(epoch, lr):
    if 0 < epoch < 12:
        return lr * 0.95
    return lr

scheduler_clbk = tf.keras.callbacks.LearningRateScheduler(scheduler)
plateu_clbk = tf.keras.callbacks.ReduceLROnPlateau(monitor='val_loss', factor=0.2, patience=5, min_lr=0.001)
earlystop_clbk = tf.keras.callbacks.EarlyStopping(monitor='loss', patience=3)

history = model.fit(x=x_train,y=y_train, validation_data=(x_test, y_test), epochs=epochs, callbacks=[scheduler_clbk, plateu_clbk, earlystop_clbk], shuffle=True)

model.save('/content/drive/MyDrive/ML/NAM_1959HW/output/'+name+'/'+name+'.h5')
model.save('/content/drive/MyDrive/ML/NAM_1959HW/output/'+name+'/'+name+'_data.h5',include_optimizer=False)

In [ ]:
model.save('models/'+name+'/'+name+'.h5')

# Run Prediction #################################################
print("Running prediction..")

y_the_rest, y_last_part = np.split(y_all, [int(len(y_all)*train_size)])
x_the_rest, x_last_part = np.split(x_all, [int(len(x_all)*train_size)])

prediction = model.predict(x_last_part)

save_wav('models/'+name+'/y_pred.wav', prediction)
save_wav('models/'+name+'/x_test.wav', x_last_part)
save_wav('models/'+name+'/y_test.wav', y_last_part)

In [ ]:
model = tf.keras.models.load_model('/content/drive/MyDrive/ML/NAM_1959HW/output/test/test_data.h5')

In [ ]:
save_wav('models/'+name+'/y_train.wav', y_last_part)

In [ ]:
plt.plot(y_train[0].flatten())

In [ ]:
clear_session()
model = tf.keras.models.load_model('models/my_model.h5')

In [ ]:
reload(ExportNNM)
from ExportNNM import *

export_model_to_nnm('/content/drive/MyDrive/ML/NAM_1959HW/output/'+name+'/'+name+'.nnm',1,0,model,1)
export_model_to_nnm('models/'+name+'/'+name+'.nnm',1,0,model,1)

In [ ]:
c1 = model.layers[0].get_weights()[0].flatten()

fig, axs = plt.subplots(nrows=2, ncols=1, figsize=(7, 7))
axs[0].plot(c1);
axs[1].magnitude_spectrum(c1, in_rate, scale='dB');

In [ ]:
plt.plot(c1);

In [ ]:
plt.plot(MPT(c1));